In [26]:
import csv
import random
import time
import json
import copy


import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = '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'
os.environ['LANGSMITH_PROJECT'] = 'pr-wooden-hedgehog-65'
os.environ['OPENAI_API_KEY'] = 'sk-g4MycviIFf8Lad0v5zVKPljDpueIDBLuqIC1nsDAiRnLZVKg'
os.environ['OPENAI_API_BASE'] = 'https://api2.aigcbest.top/v1'

from typing import Any, List, Dict

# LangChain 和 LangSmith 的相关导入
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ChatMessage

def generate_from_openai_chat_completion_new(
    messages: List[Dict[str, str]],
    model: str,
    temperature: float,
    max_tokens: int,
    top_p: float,
    context_length: int, # 这个参数在ChatOpenAI中不直接使用，但为了保持签名一致而保留
    stop_token: str | None = None,
) -> str:
    """
    使用 LangChain 的 ChatOpenAI 生成聊天回复，并支持 LangSmith 追踪。
    函数签名与旧版完全兼容。
    """
    if "OPENAI_API_KEY" not in os.environ and "api_key" not in locals():
        raise ValueError("OPENAI_API_KEY environment variable must be set.")

    # 1. 初始化 LangChain 的 ChatOpenAI 客户端
    # 将旧函数的参数映射到 ChatOpenAI 的构造函数中
    llm = ChatOpenAI(
        # 从环境变量获取 key 和 base_url，提供默认值
        api_key=os.environ.get("OPENAI_API_KEY"),
        base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1"),
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        stop=[stop_token] if stop_token else None,
        model_kwargs={}, # 如果没有其他高级参数，可以为空字典或直接省略
        # 内置重试机制，替代旧的装饰器
        max_retries=3,
    )

    # 2. 将输入的字典列表转换为 LangChain 的消息对象
    # 这是一个必要的适配步骤
    langchain_messages = []
    for msg in messages:
        role = msg.get("role")
        content = msg.get("content", "")
        # 从字典中获取 name，如果不存在则为 None
        name = msg.get("name")

        if role == "user":
            langchain_messages.append(HumanMessage(content=content, name=name))
        elif role == "assistant":
            langchain_messages.append(AIMessage(content=content, name=name))
        elif role == "system":
            # 这会正确处理您提供的 few-shot 示例
            langchain_messages.append(SystemMessage(content=content, name=name))
        else:
            # 为其他可能的角色（如 'tool'）提供一个通用处理
            langchain_messages.append(ChatMessage(role=role, content=content))

    # 3. 调用 LLM 并获取结果
    response = llm.invoke(langchain_messages)

    # 4. 提取并返回内容，保持与旧函数相同的字符串输出
    return response.content

In [27]:
system_prompt = """You are an Expert Data Synthesizer specializing in Reverse Task Synthesis for Web Agents.

Your Task:
You will be provided with a Fixed Trajectory consisting of a sequence of steps (Page Description, Action, Intent) and the Final Page's Accessibility (A11y) Tree.
Your goal is to synthesize two critical components that make this trajectory look like a coherent, purposeful interaction:
1.  **A High-Level Instruction:** The user's original command that perfectly motivates the observed actions.
2.  **A Complete Answer:** The final response the agent would generate to conclude the task, proving the instruction has been fulfilled.

---

### INPUT DATA
You will receive:
1.  `Trajectory`: A list of steps. Each step includes:
    *   `Page Description`: What the page looked like before the action.
    *   `Action`: The specific operation performed (e.g., click, type).
    *   `Local Intent`: The immediate purpose of that specific action.
2.  `Final A11y Tree`: The raw text representation of the webpage *after* the final action.

### OUTPUT REQUIREMENTS

#### 1. High-Level Instruction
*   Goal: Synthesize a natural language instruction that implies the entire sequence of actions was necessary.
*   Criteria:
    *   Abstract & Goal-Oriented: Do not list the steps (e.g., "Click A then B"). Instead, describe the *objective* (e.g., "Find the cheapest coffee maker").
    *   Consistency: The instruction must logically lead to the final state.
    *   No IDs: Never mention element IDs (e.g., [74]) in the instruction.

#### 2. Complete Answer
*   Goal: Simulate the agent's Final Thought Process (Chain-of-Thought) that validates the result and issues the stop command.
*   Format:
    *   MUST start with: `"Let's think step-by-step."`
    *   MUST end with: `In summary, the next action I will perform is ` + ```stop [content]```.
    *   NO text is allowed after the code block.
*   Logic Types (Choose the one that fits the Final A11y Tree):
    *   Type A (Information Seeking): If the instruction was to find specific text (e.g., a price, a date, a comment), extract this answer **verbatim** from the provided `Final A11y Tree`.
    *   Type B (Task Completion/Navigation): If the instruction was to perform an action (e.g., "Navigate to...", "Log in...", "Change settings"), and the final state confirms this (or the second-to-last action completed it), the answer should summarize what was accomplished (e.g., "I have successfully navigated to...").

---

### RESPONSE FORMAT
You must return a single JSON object with the following structure:

{
  "Instruction": "The synthesized high-level user command.",
  "Answer": "Let's think step-by-step. [Reasoning about how the previous actions led to the current state and extracting the final answer or confirming completion].\n\nIn summary, the next action I will perform is ```stop [The extracted answer or completion message]```"
}

### EXAMPLES
Trajectory: User searched for "Python", clicked "Downloads", saw version "3.11.4" in the final A11y tree.
Output:{
  "Instruction": "Check the official Python website to find the version number of the latest release available for download.",
  "Answer": "Let's think step-by-step. I have navigated to the Python website and accessed the Downloads section. Looking at the current page's accessibility tree, I can clearly see the text 'Download Python 3.11.4'. This confirms the latest version number. \n\nIn summary, the next action I will perform is ```stop [3.11.4]```"
}

Trajectory: User clicked "Settings", then clicked "Dark Mode". The final page shows the settings menu.
Output:{
  "Instruction": "Go to the application settings and enable the dark mode theme.",
  "Answer": "Let's think step-by-step. I navigated to the settings page and clicked on the 'Dark Mode' toggle. The action has been executed. Since the goal was to change this setting and I have performed the necessary interaction, the task is complete.\n\nIn summary, the next action I will perform is ```stop [Dark mode enabled]```"
}"""

In [28]:
with open("/home/zjusst/qms/webarena/result_stage_3_generation/random_selected_trajs.json", "r") as f:
    random_selected_trajs = json.load(f)

selected_trajectory = random_selected_trajs[40]["selected_trajectory"]
selected_trajectory


[["('812_20.json', 14, 1, 'http://reddit.com/f/relationship_advice')",
  "click [4924] where [4924] is button 'Sort by: Hot' hasPopup: menu expanded: False",
  'The user wants to change how posts are sorted to view them in a different order.',
  "('812_20.json', 15, 1, 'http://reddit.com/f/relationship_advice')"],
 ["('812_20.json', 15, 1, 'http://reddit.com/f/relationship_advice')",
  "click [6670] where [6670] is link 'New'",
  "The user wants to view the latest posts in the 'relationship_advice' forum by changing the sorting order to 'New'.",
  "('812_20.json', 16, 1, 'http://reddit.com/f/relationship_advice/new')"],
 ["('812_20.json', 16, 1, 'http://reddit.com/f/relationship_advice/new')",
  "click [6956] where [6956] is button 'Upvote'",
  'The user intends to express approval or agreement with the content of a specific post by upvoting it.',
  "('812_20.json', 17, 1, 'http://reddit.com/f/relationship_advice/new')"]]

In [29]:
def get_origin_traj_json(filename, index, folder_path="/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs"):
    with open(os.path.join(folder_path, filename), "r") as f:
        origin_traj_json = json.load(f)
    return origin_traj_json[index]

#get_origin_traj_json("812_1.json", 0)

In [30]:
def selected_trajectory_to_user_prompt(selected_trajectory):
    user_prompt = ""
    traj_len = len(selected_trajectory)
    for i in range(traj_len-1):
        filename, index, _, _ = eval(selected_trajectory[i][0])
        orgin_json = get_origin_traj_json(filename, index)

        page_description = orgin_json["web_desc_before"]
        action = orgin_json["action_str"]
        intention = orgin_json["intention"]
        user_prompt += f"""Page{i+1}:
Page Description: {page_description}
Action: {action}
Local Intent: {intention}
        
"""
    filename, index, _, _ = eval(selected_trajectory[-1][0])
    orgin_json = get_origin_traj_json(filename, index)
    final_a11y_tree = orgin_json["a11y_before"]
    user_prompt += f"""Page{traj_len}:
Final Page Accessibility Tree: {final_a11y_tree}
    """

    user_prompt += f"""
Now, please generate the json include the high-level instruction and the complete answer for the above trajectory.
    """
    return user_prompt

selected_trajectory = random_selected_trajs[88]["selected_trajectory"]
print(selected_trajectory_to_user_prompt(selected_trajectory))

Page1:
Page Description: A wiki page on Reddit showing an article about exploring new features, with navigation links including 'Home', 'Wiki', and 'Forums', and a sidebar with editing tools.
Action: click [6217] where [6217] is link 'Page history'
Local Intent: The user wants to view the edit history of the current wiki page to see past changes or compare versions.
        
Page2:
Page Description: A wiki page showing the history of edits for a specific page titled 'index'. The page includes a table with revision details, a button for comparing revisions, and a link to edit the page.
Action: click [6443] where [6443] is link 'Edit page'
Local Intent: The user wants to edit the 'index' wiki page, transitioning from the history view to the editing interface.
        
Page3:
Page Description: A wiki editing page where the user can edit the 'index' page by entering a title and body, with options to save changes and view formatting help. The page includes navigation links and information a

In [31]:
example_messages_with_name = [
    {
        'role': 'system',
        'content': system_prompt
    },
    {
        'role': 'user',
        'name': 'user',
        'content': selected_trajectory_to_user_prompt(selected_trajectory)
    }
]

result = generate_from_openai_chat_completion_new(
    messages=example_messages_with_name,
    model="gpt-4o",
    temperature=0.7,
    max_tokens=2000,
    top_p=1.0,
    context_length=8192,
    stop_token=None
)

In [32]:
print(result)

```json
{
  "Instruction": "Access and update the 'index' wiki page on Reddit, then save the changes to ensure the latest version is published.",
  "Answer": "Let's think step-by-step. I started by navigating to the 'Page history' section to review past edits of the 'index' wiki page. Then, I proceeded to the editing interface to make the necessary updates. After reviewing the content, I scrolled down to ensure all options were visible and finally saved the changes. Upon saving, the wiki page was successfully updated and reflects the new content. The final page confirms that the updates I made are now live with the title 'Sample TitleExploring New Features' and the body text as expected. 

In summary, the next action I will perform is ```stop [Sample TitleExploring New Features]```"
}
```


In [34]:
def extract_instruction_and_answer(result):
    """
    解析类似```json
    {
    "Instruction": "Access and update the 'index' wiki page on Reddit, then save the changes to ensure the latest version is published.",
    "Answer": "Let's think step-by-step. I started by navigating to the 'Page history' section to review past edits of the 'index' wiki page. Then, I proceeded to the editing interface to make the necessary updates. After reviewing the content, I scrolled down to ensure all options were visible and finally saved the changes. Upon saving, the wiki page was successfully updated and reflects the new content. The final page confirms that the updates I made are now live with the title 'Sample TitleExploring New Features' and the body text as expected. 

    In summary, the next action I will perform is ```stop [Sample TitleExploring New Features]```"
    }
    ```
    """

    if not result.startswith("```json"):
        # 直接解析
        try:
            return json.loads(result)
        except json.JSONDecodeError:
            raise ValueError(f"Invalid JSON format in result: {result}")
    else:
        # 从```json```中解析
        json_str = result.split("```json")[1].split("```")[0]
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            raise ValueError(f"Invalid JSON format in result: {result}")

extract_instruction_and_answer(result)

ValueError: Invalid JSON format in result: ```json
{
  "Instruction": "Access and update the 'index' wiki page on Reddit, then save the changes to ensure the latest version is published.",
  "Answer": "Let's think step-by-step. I started by navigating to the 'Page history' section to review past edits of the 'index' wiki page. Then, I proceeded to the editing interface to make the necessary updates. After reviewing the content, I scrolled down to ensure all options were visible and finally saved the changes. Upon saving, the wiki page was successfully updated and reflects the new content. The final page confirms that the updates I made are now live with the title 'Sample TitleExploring New Features' and the body text as expected. 

In summary, the next action I will perform is ```stop [Sample TitleExploring New Features]```"
}
```